# Fastscapelib `NumbaEroderType` example

In [ ]:
import matplotlib.pyplot as plt
import numba as nb
import numpy as np

from fastscapelib.eroders import FlowKernelEroder
import fastscapelib as fs


grid = fs.RasterGrid.from_length([201, 301], [5e4, 7.5e4], fs.NodeStatus.FIXED_VALUE)
# flow_graph = fs.FlowGraph(grid, [fs.MultiFlowRouter()])
flow_graph = fs.FlowGraph(grid, [fs.SingleFlowRouter(), fs.MSTSinkResolver()])

rng = np.random.Generator(np.random.PCG64(1234))

init_elevation = rng.uniform(0, 5, size=grid.shape)
drainage_area = np.empty_like(init_elevation)

uplift_rate = np.full_like(init_elevation, 1e-3)
uplift_rate[[0, -1], :] = 0.0
uplift_rate[:, [0, -1]] = 0.0
flow_graph.update_routes(init_elevation)

class NumbaSplEroder(FlowKernelEroder):
    spec = dict(
        elevation=nb.float64[::1],
        erosion=nb.float64[::1],
        drainage_area=nb.float64[::1],
        k_coef=(nb.float64, 2e-4),
        area_exp=(nb.float64,0.4),
        slope_exp=(nb.float64, 1.),
        dt=(nb.float64, 2e4),
    )

    outputs = ["erosion"]
    
    apply_dir=fs.FlowGraphTraversalDir.BREADTH_UPSTREAM
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.kernel_data.bind(erosion=np.zeros(flow_graph.size))
        
    def erode(self, elevation, drainage_area, dt):
        self.set_kernel_data(elevation=elevation, drainage_area=drainage_area, dt=dt)
        self.kernel_data.erosion.fill(0.)
        super().erode()
        return self.kernel_data.erosion
        
    @staticmethod
    def eroder_kernel(node):
        dt = node.dt

        r_count = node.receivers.count
        if r_count == 1 and node.receivers.distance[0] == 0.0:
            return
    
        elevation_flooded = np.finfo(np.double).max
    
        for r in range(r_count):
            irec_elevation_next = node.receivers.elevation[r] - node.receivers.erosion[r]
    
            if irec_elevation_next < elevation_flooded:
                elevation_flooded = irec_elevation_next
    
        if node.elevation <= elevation_flooded:
            return
    
        eq_num = node.elevation
        eq_den = 1.0
    
        for r in range(r_count):
            irec_elevation = node.receivers.elevation[r]
            irec_elevation_next = irec_elevation - node.receivers.erosion[r]
    
            if irec_elevation > node.elevation:
                continue
    
            irec_weight = node.receivers.weight[r]
            irec_distance = node.receivers.distance[r]
    
            factor = (
                node.k_coef * node.dt * np.power(node.drainage_area * irec_weight, node.area_exp)
            )
            factor /= irec_distance
            eq_num += factor * irec_elevation_next
            eq_den += factor
    
        elevation_updated = eq_num / eq_den
    
        if elevation_updated < elevation_flooded:
            elevation_updated = elevation_flooded + np.finfo(np.double).tiny
    
        node.erosion = node.elevation - elevation_updated
        
elevation = init_elevation.ravel().copy()
erosion = np.zeros(flow_graph.size)
drainage_area = np.ones(flow_graph.size)

In [ ]:
dt = 2e4
nsteps = 50
uplift = dt * uplift_rate.ravel()

spl_eroder = NumbaSplEroder(flow_graph, max_receivers=10)

fs_spl_eroder = fs.SPLEroder(
    flow_graph,
    k_coef=2e-4,
    area_exp=0.4,
    slope_exp=1.,
    tolerance=1e-5,
)

def run_simulation(eroder):
    elevation = init_elevation.copy().ravel()
    drainage_area = np.empty_like(elevation)

    for step in range(nsteps):
        # uplift (no uplift at fixed elevation boundaries)
        uplifted_elevation = elevation + uplift
   
        # flow routing
        flow_graph.update_routes(uplifted_elevation)

        # flow accumulation (drainage area)
        flow_graph.accumulate(drainage_area, 1.0)

        erosion = eroder.erode(uplifted_elevation, drainage_area, dt)

        # update topography
        elevation = uplifted_elevation - erosion.ravel()

    return elevation.reshape(grid.shape)

In [ ]:
spl_eroder.erode(elevation=elevation, drainage_area=drainage_area, dt=dt);

In [ ]:
for i in [1, 2, 4, 8]:
    spl_eroder.n_threads = i
    %timeit -r 10 -n 10 spl_eroder.erode(elevation=elevation, drainage_area=drainage_area, dt=dt)

In [ ]:
for i in [1, 2, 4, 8, 16]:
    spl_eroder.n_threads = i
    %timeit -r 1 -n 1 run_simulation(spl_eroder)

In [ ]:
# efficiency of the parallel execution depends to the kernel size, the current flow graph and the kernel execution order
for i in [1, 2, 4, 8]:
    spl_eroder.n_threads = i
    %timeit -r 10 -n 10 spl_eroder.erode(elevation=elevation, drainage_area=drainage_area, dt=dt)

In [ ]:
%timeit -r 1 -n 1 run_simulation(fs_spl_eroder)

In [ ]:
%timeit -r 10 -n 10 fs_spl_eroder.erode(elevation, drainage_area, dt)

### Benchmark Results

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))

spl_eroder._kernel.kernel.n_threads = 1
axes[0, 0].imshow(run_simulation(spl_eroder).reshape(grid.shape))
spl_eroder._kernel.kernel.n_threads = 4
axes[0, 1].imshow(run_simulation(spl_eroder).reshape(grid.shape))
spl_eroder._kernel.kernel.n_threads = 8
axes[1, 0].imshow(run_simulation(spl_eroder).reshape(grid.shape))

axes[1, 1].imshow(run_simulation(fs_spl_eroder));